In [1]:
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
import warnings
warnings.filterwarnings('ignore')

In [2]:
llm = GoogleGenerativeAI(model="gemini-pro")

In [3]:
from langchain import PromptTemplate
from langchain.llms import GooglePalm
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [4]:
BASELINE_PROMPT = """Answer the below question which is asking for a concise factual answer. NO ADDITIONAL DETAILS.

Question: {query}

Answer:"""


# Chain to generate initial response
baseline_response_prompt_template = PromptTemplate.from_template(BASELINE_PROMPT)
baseline_response_chain = baseline_response_prompt_template | llm | StrOutputParser()

In [5]:
VERIFICATION_QUESTION_TEMPLATE = """Your task is to create a verification question based on the below question provided.
Example Question: Who wrote the book 'God of Small Things' ?
Example Verification Question: Was book [God of Small Things] written by [writer]? If not who wrote [God of Small Things] ? 
Explanation: In the above example the verification question focused only on the ANSWER_ENTITY (name of the writer) and QUESTION_ENTITY (book name).
Similarly you need to focus on the ANSWER_ENTITY and QUESTION_ENTITY from the actual question and generate verification question.

Actual Question: {query}

Final Verification Question:"""


# Chain to generate a question template for verification answers
verification_question_template_prompt_template = PromptTemplate.from_template(VERIFICATION_QUESTION_TEMPLATE)
verification_question_template_chain = verification_question_template_prompt_template | llm | StrOutputParser()

In [6]:
VERIFICATION_QUESTION_PROMPT= """Your task is to create a series of verification questions based on the below question, the verfication question template and baseline response.
Example Question: Who wrote the book 'God of Small Things' ?
Example Verification Question Template: Was book [God of Small Things] written by [writer]? If not who wrote [God of Small Things]?
Example Baseline Response: Jhumpa Lahiri
Example Verification Question: 1. Was God of Small Things written by Jhumpa Lahiri? If not who wrote God of Small Things ?


Explanation: In the above example the verification questions focused only on the ANSWER_ENTITY (name of the writer) and QUESTION_ENTITY (name of book) based on the template and substitutes entity values from the baseline response.
Similarly you need to focus on the ANSWER_ENTITY and QUESTION_ENTITY from the actual question and substitute the entity values from the baseline response to generate verification questions.

Actual Question: {query}
Baseline Response: {base_response}
Verification Question Template: {verification_question_template}

Final Verification Questions:"""


# Chain to generate the verification questions
verification_question_generation_prompt_template = PromptTemplate.from_template(VERIFICATION_QUESTION_PROMPT)
verification_question_generation_chain = verification_question_generation_prompt_template | llm | StrOutputParser()

In [7]:
from langchain.agents import ConversationalChatAgent, AgentExecutor
from langchain.tools import DuckDuckGoSearchResults

#create search agent
search = DuckDuckGoSearchResults()
tools = [search]
custom_system_message = "Assistant assumes no knowledge & relies on internet search to answer user's queries."
max_agent_iterations = 5
max_execution_time = 10

chat_agent = ConversationalChatAgent.from_llm_and_tools(
    llm=llm, tools=tools, system_message=custom_system_message
)
search_executor = AgentExecutor.from_agent_and_tools(
    agent=chat_agent,
    tools=tools,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    max_iterations=max_agent_iterations,
    max_execution_time = max_execution_time
)

# chain to execute verification questions
verification_chain = RunnablePassthrough.assign(
    split_questions=lambda x: x['verification_questions'].split("\n"), # each verification question is passed one by one factored approach
) | RunnablePassthrough.assign(
    answers = (lambda x: [{"input": q,"chat_history": []} for q in x['split_questions']])| search_executor.map() # search executed for each question independently
) | (lambda x: "\n".join(["Question: {} Answer: {}\n".format(question, answer['output']) for question, answer in zip(x['split_questions'], x['answers'])]))# Create final refined response

In [8]:
FINAL_ANSWER_PROMPT= """Given the below `Original Query` and `Baseline Answer`, analyze the `Verification Questions & Answers` to finally provide the refined answer.
Original Query: {query}
Baseline Answer: {base_response}

Verification Questions & Answer Pairs:
{verification_answers}

Final Refined Answer:"""


# Chain to generate the final answer
final_answer_prompt_template = PromptTemplate.from_template(FINAL_ANSWER_PROMPT)
final_answer_chain = final_answer_prompt_template | llm | StrOutputParser()

In [9]:
chain = RunnablePassthrough.assign(
    base_response=baseline_response_chain
) |  RunnablePassthrough.assign(
    verification_question_template=verification_question_template_chain
) | RunnablePassthrough.assign(
    verification_questions=verification_question_generation_chain
) | RunnablePassthrough.assign(
    verification_answers=verification_chain
) | RunnablePassthrough.assign(
    final_answer=final_answer_chain
)

response = chain.invoke({"query": "Who wrote the book 'Economics of Small Things' ?"})

In [10]:
from pprint import pprint 
pprint(response)

{'base_response': 'Warren Buffett',
 'final_answer': 'The Economics of Small Things was written by Dr. Sudipta '
                 'Sarangi, a distinguished economist and Professor at Virginia '
                 'Tech, USA.',
 'query': "Who wrote the book 'Economics of Small Things' ?",
 'verification_answers': 'Question: 1. Was Economics of Small Things written '
                         'by Arundhati Roy? If not who wrote Economics of '
                         'Small Things? Answer: The Economics of Small Things '
                         'was written by Dr. Sudipta Sarangi, a distinguished '
                         'economist and Professor at Virginia Tech, USA.\n',
 'verification_question_template': 'Was book [Economics of Small Things] '
                                   'written by [Arundhati Roy]? If not who '
                                   'wrote [Economics of Small Things] ?',
 'verification_questions': '1. Was Economics of Small Things written by '
                     

In [11]:
print(response['final_answer'])

The Economics of Small Things was written by Dr. Sudipta Sarangi, a distinguished economist and Professor at Virginia Tech, USA.


In [19]:
response = chain.invoke({"query": "Who is Charana H U? does he creates  youtube videos in kannada?, which is his latest llm video"})
print(response['final_answer'])

Task was destroyed but it is pending!
task: <Task pending name='Task-31' coro=<AsyncDDGS.__aexit__() running at /Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/duckduckgo_search/duckduckgo_search_async.py:46>>
Task was destroyed but it is pending!
task: <Task pending name='Task-15' coro=<AsyncCurl._force_timeout() running at /Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/curl_cffi/aio.py:168> wait_for=<Future pending cb=[Task.__wakeup()]>>
Task was destroyed but it is pending!
task: <Task pending name='Task-36' coro=<AsyncDDGS.__aexit__() running at /Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/duckduckgo_search/duckduckgo_search_async.py:46>>
Task was destroyed but it is pending!
task: <Task pending name='Task-18' coro=<AsyncCurl._force_timeout() running at /Users/charan/VS_Code/GitHub/Advanced_RAG/venv/lib/python3.11/site-packages/curl_cffi/aio.py:168> wait_for=<Future pending cb=[Task.__wakeup()]>

Yes, Charana H U creates YouTube videos in Kannada. His most recent video is titled 'Sandak Nam | සඳක් නම් | Rohan Shantha Bulegoda | Charana Beats'.


In [13]:
pprint(response)

{'base_response': '',
 'final_answer': 'I apologize, but I do not have the information necessary to '
                 'answer your question. You may want to try searching for the '
                 'answer using a search engine.',
 'query': 'Who is Charana H U? does he creates  youtube videos in kannada?, '
          'which is his latest llm video',
 'verification_answers': 'Question: 1. Is Charana H U the creator of the '
                         'youtube video [any video title]? If not, who is the '
                         'creator of the youtube video [any video title]? '
                         'Answer: I was unable to find the answer to your '
                         'question in the context you provided.\n'
                         '\n'
                         'Question: 2. Is Charana H U the creator of the '
                         'youtube video [any video title]? If not, who is the '
                         'creator of the youtube video [any video title]? '
            

In [14]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchRun()
search.run("Charan H U?")

'Written by Charan H U. 27 Followers. Applied AI Engineer | Internet Content Creator | Freelancer | Farmer | Student. Follow. More from Charan H U. Charan H U. Softmax and cross-entropy for multi-class classification. Let\'s say you\'re trying to classify animals in a jungle. You have pictures of different animals like lions, tigers, bears and ... Charan H U Applied AI Engineer at IBM | Responsible Transparent AI Application Builder | Internet Content Creator Additionally, adopting standardized methods for explainability, promoting ... Charan Ranganath fitting an EEG cap (to read electrical signals from the brain) on a volunteer in 2013. UC Davis I think everyone has seemingly inexplicable memories that stick with us. Charan Ranganath is a professor of psychology and neuroscience at UC Davis. This essay was adapted for the Los Angeles Times from the author\'s forthcoming book, "Why We Remember: Unlocking ... Memory is a quirky thing, writes Ranganath, director of the Dynamic Memory Lab

In [15]:
search = DuckDuckGoSearchResults()
search.run("Charan H U?")

'[snippet: Written by Charan H U. 27 Followers. Applied AI Engineer | Internet Content Creator | Freelancer | Farmer | Student. Follow. More from Charan H U. Charan H U. Softmax and cross-entropy for multi-class classification. Let\'s say you\'re trying to classify animals in a jungle. You have pictures of different animals like lions, tigers, bears and ..., title: Converting Pandas DataFrame into a Dataset and Pushing to Hugging Face ..., link: https://charanhu.medium.com/converting-pandas-dataframe-into-a-dataset-and-pushing-to-hugging-face-146e2ccac38d], [snippet: Charan Ranganath fitting an EEG cap (to read electrical signals from the brain) on a volunteer in 2013. UC Davis I think everyone has seemingly inexplicable memories that stick with us., title: A Leading Memory Researcher Explains How to Make Precious Moments Last ..., link: https://www.nytimes.com/interactive/2024/02/04/magazine/charan-ranganath-interview.html], [snippet: Charan Ranganath recently wrote an op-ed about Pre

In [16]:
search = DuckDuckGoSearchResults(backend="youtube")
search.run("Charan H U?")

''

In [17]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(time="d")

In [18]:
search = DuckDuckGoSearchRun()
search.run("'Who is Charan H U'?")

'Charan Kaur, also known as Charanjeet Kaur Sidhu, is a name that has recently made headlines for two reasons. One, she is the mother of the late Punjabi rap ... Delhi University (2013 - 2016). • PG Diploma in Journalism (English), IIMC Delhi (2012). Experience • 6 Month Digital media Intern at Outlook India, New Delhi (2016). CCSHAU Hisar Courses and Fees 2023. The modes of Chaudhary Charan Singh Haryana Agricultural University course admissions 2023 are online and offline. Admission to various CCSHAU Hisar Courses is based on entrance exams and merit. At the PG level, Full-time M.Sc., M.F.Sc., M.Tech., and MBA programmes are available. Charan Singh (born Dec. 23, 1902, Noorpur, Uttar Pradesh, India—died May 29, 1987, New Delhi) was an Indian politician who served briefly as prime minister (1979-80) and was known for his land and agricultural reform initiatives.. Singh became a lawyer and in 1929 joined the Indian National Congress movement. He was jailed several times in the struggle